<a href="https://colab.research.google.com/github/PrattSAVI/ACS/blob/master/NYC_CensusTable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Retirieveing Data from Census API
This notebook walks through the steps necessary to access Census data through Census API. Currently this notebook only works for census tracts and block groups within NYC.

Steps:
* Setup
* Import Geometry
* Retrieve information
* Format tables
* Merge data with shapefiles for mapping. 
* Visualize the information

Please get your API key from Census.gov, by [registering here](https://api.census.gov/data/key_signup.html).


In [ ]:
#@title ### Setup
#@markdown Install necessary dependencies
!pip install geopandas
!pip install seaborn 
!pip install plotly

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [40]:
#@title ### Enter Parameters
#@markdown Define Geometry Resolution, Table/Group Name, Product and Year

Table_name = "B19013" #@param ["B19013", "S1701", "S2201"] {allow-input: true}
Product = "acs5" #@param[ "acs5", "acs1" ]
Year = 2018 #@param[2019,2018,2017,2016,2015,2014,2013,2012,2011,2010]
api_key = '155e59c93c8216ac7e2a18584db8417bf7279f7f' #@param
Analysis_Geometry = "Census Tract"  #@param ['Census Tract', 'Block Group' ]

if Analysis_Geometry == 'Census Tract':
  Analysis_Geometry = 'tract'
elif Analysis_Geometry == 'Block Group':
  Analysis_Geometry = 'block%20group'

if Analysis_Geometry == 'tract':
  gdf = gpd.read_file( r'https://raw.githubusercontent.com/PrattSAVI/ACS/master/data/2018_NYC_TRACT.geojson')
if Analysis_Geometry == 'block%20group':
  gdf = gpd.read_file( r'https://raw.githubusercontent.com/PrattSAVI/ACS/master/data/2018_NYC_BG.geojson')

gdf.plot(
    facecolor="none",
    edgecolor="black",
    lw = 0.1,
)
plt.tick_params(left=False , bottom = False , labelleft=False, labelbottom=False)
sns.despine( left = True , bottom = True)
plt.show()

In [134]:

bg = pd.DataFrame() #Empty DF to store all info

path_group = r'https://api.census.gov/data/2018/acs/acs5?get=NAME,group(' + Table_name + ')&for=' + Analysis_Geometry + ':*&in=state:36%20county:' + str(i) + '&key=' + api_key
path_table = r'https://api.census.gov/data/2018/acs/acs5/subject?get=NAME,group(' + Table_name + ')&for=' + Analysis_Geometry + ':*&in=state:36%20county:' + str(i) + '&key=' + api_key

if Table_name[0] == "S":
  path = path_table
if Table_name[0] == "B":
  path = path_group

for i in gdf['COUNTYFP'].unique(): #Retrieve Block Group info for each County
  path = r'https://api.census.gov/data/2018/acs/acs5?get=NAME,group(' + Table_name + ')&for=' + Analysis_Geometry + ':*&in=state:36%20county:' + str(i) + '&key=' + api_key
  #print ('API URL: ' , path)
  df = pd.read_json( path )
  df.columns = df.loc[0]
  df = df.iloc[1:]

  bg = bg.append( df )

bg.sample(3)

,NAME,GEO_ID,B19013_001E,B19013_001M,NAME,B19013_001MA,B19013_001EA,state,county,tract
389,"Census Tract 208, Queens County, New York",1400000US36081020800,82813,23698,"Census Tract 208, Queens County, New York",None,None,36,081,020800
242,"Census Tract 299, New York County, New York",1400000US36061029900,20338,4680,"Census Tract 299, New York County, New York",None,None,36,061,029900
587,"Census Tract 922, Kings County, New York",1400000US36047092200,28792,24012,"Census Tract 922, Kings County, New York",None,None,36,047,092200


In [135]:
#Remove all values keeping the columns to be converted to numeric

data_cols = bg.columns[ bg.columns.str.contains( Table_name ) ]
for _ in data_cols.tolist():
  bg.loc[ bg[_] ==  '(X)' , _ ] = None
  bg.loc[ bg[_] ==  '*' , _ ] = None
  bg.loc[ bg[_] ==  '**' , _ ] = None
  bg.loc[ bg[_] ==  '***' , _ ] = None
  bg.loc[ bg[_] ==  '-' , _ ] = None
  bg.loc[ bg[_] ==  '250,000+' , _ ] = 250001
  bg.loc[ bg[_] ==  -666666666 , _ ] = None
  bg.loc[ bg[_] ==  -222222222 , _ ] = None
  bg[_] = bg[_].astype( float )

bg.sample(3)

,NAME,GEO_ID,B19013_001E,B19013_001M,NAME,B19013_001MA,B19013_001EA,state,county,tract
505,"Census Tract 398, Kings County, New York",1400000US36047039800,54850.0,23281.0,"Census Tract 398, Kings County, New York",NaN,NaN,36,047,039800
361,"Census Tract 525, Kings County, New York",1400000US36047052500,34091.0,11321.0,"Census Tract 525, Kings County, New York",NaN,NaN,36,047,052500
132,"Census Tract 220, New York County, New York",1400000US36061022000,66578.0,10864.0,"Census Tract 220, New York County, New York",NaN,NaN,36,061,022000


In [136]:
bg_clean = bg #New df for the cleaned data

#All available Column names
cols = pd.read_json( r'https://api.census.gov/data/2018/acs/acs5/variables.json' )

#Use label column names if available
label = []
for c in bg_clean.columns.tolist():
    temp = cols[ cols.index == c ]
    if len(temp) != 0:
      col_name =  temp['variables'][0]['label'] 
      label.append( col_name )
    else:
      label.append( c )
bg_clean.columns = label 

#Clean data
bg_clean = bg_clean.dropna( how = 'all' , axis = 1) #Remove columns if there is no value  

bg_clean = bg_clean.loc[:,~bg_clean.columns.duplicated()] #Remove duplicate NAME Columns
bg_clean = bg_clean[ bg_clean.columns[ ~bg_clean.columns.str.contains('EA') ] ] #Remove Annotations Columns
bg_clean = bg_clean[ bg_clean.columns[ ~bg_clean.columns.str.contains('MA') ] ] #Remove Annotations Columns

bg_clean.sample( 3 )

,NAME,Geography,Estimate!!Median household income in the past 12 months (in 2018 inflation-adjusted dollars),B19013_001M,state,county,tract
10,"Census Tract 248, Bronx County, New York",1400000US36005024800,58553.0,12077.0,36,005,024800
4,"Census Tract 371, Bronx County, New York",1400000US36005037100,27925.0,8082.0,36,005,037100
148,"Census Tract 74, Bronx County, New York",1400000US36005007400,38490.0,6441.0,36,005,007400


In [137]:
gdf['GEOID_2'] = '1400000US' + gdf['GEOID']
gdf = gdf.join( bg_clean.set_index('Geography') , on = 'GEOID_2' , rsuffix = '_D' )

gdf.head(3)

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,GEOID_2,NAME_D,Estimate!!Median household income in the past 12 months (in 2018 inflation-adjusted dollars),B19013_001M,NAME_D,B19013_001MA,B19013_001EA,state,county,tract,NAME_D,Estimate!!Median household income in the past 12 months (in 2018 inflation-adjusted dollars)_D,B19013_001M_D,state_D,county_D,tract_D,NAME_D,Estimate!!Median household income in the past 12 months (in 2018 inflation-adjusted dollars)_D,B19013_001M_D,state_D,county_D,tract_D
0,36,081,046200,36081046200,462,Census Tract 462,G5020,S,249611,0,+40.7098547,-073.7879749,"MULTIPOLYGON (((-73.79203 40.71107, -73.79101 ...",1400000US36081046200,"Census Tract 462, Queens County, New York",53295,9902,"Census Tract 462, Queens County, New York",None,None,36,081,046200,"Census Tract 462, Queens County, New York",53295,9902,36,081,046200,"Census Tract 462, Queens County, New York",53295.0,9902.0,36,081,046200
1,36,081,045000,36081045000,450,Census Tract 450,G5020,S,172164,0,+40.7141840,-073.8047700,"MULTIPOLYGON (((-73.80782 40.71591, -73.80767 ...",1400000US36081045000,"Census Tract 450, Queens County, New York",90568,14835,"Census Tract 450, Queens County, New York",None,None,36,081,045000,"Census Tract 450, Queens County, New York",90568,14835,36,081,045000,"Census Tract 450, Queens County, New York",90568.0,14835.0,36,081,045000
2,36,081,045400,36081045400,454,Census Tract 454,G5020,S,230996,0,+40.7126504,-073.7960120,"MULTIPOLYGON (((-73.79870 40.71066, -73.79792 ...",1400000US36081045400,"Census Tract 454, Queens County, New York",45958,10460,"Census Tract 454, Queens County, New York",None,None,36,081,045400,"Census Tract 454, Queens County, New York",45958,10460,36,081,045400,"Census Tract 454, Queens County, New York",45958.0,10460.0,36,081,045400


In [167]:


import ipywidgets as widgets
from IPython.display import display

w = widgets.Dropdown(
    options= gdf.columns.tolist(),
    description='Choose Column to Visualize:',
    disabled=False,
)

def on_change(change):#Dropdown on_change event
    if change['type'] == 'change' and change['name'] == 'value':
        mapper( change['new'] , gdf )

w.observe( on_change )

print( "Choose a Column to continue!")
display(w)

import plotly.express as px

def mapper(viz_col , gdf): #Mapping is here
  fig = px.choropleth_mapbox(gdf , geojson=gdf.geometry , locations=gdf.index,
                            color= viz_col,opacity = 0.5,
                            hover_name= viz_col ,
                            center={"lat": 40.71, "lon": -74.00},zoom=9,
                            mapbox_style="carto-positron",
                            color_continuous_scale="BuPu", # Here are more colors: https://plotly.com/python/builtin-colorscales/
                          )

  fig.update_traces( marker = dict( line=dict(width=0) ) )
  #fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
  fig.update_layout(width=800,height=600)
  fig.show()


Choose a Column to continue!


Dropdown(description='Choose Column to Visualize:', options=('STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'NAME'…

ValueError: ignored

ValueError: ignored

ValueError: ignored